In [81]:
import pandas as pd
import datetime as dt
import numpy as np
import os
from oodle import config
from oodle.integrations.hooks.oodle_postgres_hook import OodlePostgresHook

In [82]:
def setup_connection():
    return OodlePostgresHook(config.REDSHIFT_SECRET).get_conn()

hpi_kc = pd.read_excel(os.getcwd() + '/HPI keeper change report 010920-300920.xlsx')

In [88]:
def format_hpi_report(hpi_kc):
    hpi_kc.drop_duplicates(subset=['VIN']).reset_index(inplace=True)
    hpi_kc.astype({'VIN' : 'str'})
    return hpi_kc
 
hpi_kc = format_hpi_report(hpi_kc)

In [85]:
def grab_portfolio():
    query = '''
    SELECT la.agreement_code,
       la.registration,
       la.VIN,
       cd.first_name,
       cd.last_name,
       lt.status_code,
       lt.capital_balance,
       lt.arrears,
       lt.days_in_arrears,
       ad.current_address__c
    FROM oodledata_loans.loan_agreements AS la
    LEFT JOIN oodledata.customer_dim AS cd
    ON cd.account_id = la.main_borrower_account_id
    LEFT JOIN oodledata_loans.loan_timeline AS lt
    ON lt.agreement_code = la.agreement_code AND date = '2020-10-29'  
    LEFT JOIN salesforce_realtime.account_defeed AS ad
    ON ad.id = la.main_borrower_account_id;
    '''
    df1 = pd.read_sql(query, setup_connection())
    df1 = df1.rename(columns={'registration': 'VRM', 'vin':'VIN'})
    df1.VIN.astype(str)
    return df1

portfolio = grab_portfolio()

In [86]:
final_df = hpi_kc.merge(portfolio, how='left', on='VIN', copy=False)
final_df = final_df.rename(columns = {'VRM_y':'Oodle VRM', 'VRM_x':'HPI VRM'})

In [80]:
final_df.to_excel(os.getcwd() + '/Populated HPI keeper changes report 010920-300920.xlsx')